Lista de TPDO(Transmision Process Data Object) y RPDO(Receive Process Data Object) definidos en el maestro y en el esclavo son:

# TPDO

Maestro
1.   COB-ID: $0x00000181$
  - Controlword right motor : $0x2021,1$
  - Velocidad Máxima right motor  RPM : $0x2021,3$
  - Torque objetivo %: $0x2021,5$
2.   COB-ID: $0x00000100$
  - Voltaje bateria V : $0x5100,1$
  - Corriente Batería A : $0x5100,2$
  - Temperatura Inversor : $0x5100,4$
3.   COB-ID: $0x00000200$
  - Corriente estator A : $0x6078,0$
  - Torque electromecanico Nm : $0x4602,12$
  - Velocidad RPM : $0x606C,0$
4.   COB-ID: $0x00000300$
  - Iq objetivo A : $0x4600,2$
  - Iq A : $0x4600,8$
  - Demanda de Torque A : $0x4602,11$
  - Uq V : $0x4600,10$
5.   COB-ID: 0x00000400
  - Voltaje acelerador V : $0x2220,0$
  - Id A : $0x460,7$
  - Id objetivo A : $0x4600,5$
  - Demanda de Torque V : $0x4602,11$

Esclavo
1.   COB-ID: $0x00000201$
  - Status word : $0x6041,0$
  - Velocidad RPM : $0x606C,0$
  - Torque Nm : $0x2021,5$
  - Torque objetivo : $0x2021,5$
2.   COB-ID: $0x00000100$
  - Voltaje bateria V : $0x5100,1$
  - Corriente Batería : $0x5100,2$
  - Temperatura Inversor : $0x5100,4$
3.   COB-ID: $0x00000200$
  - Corriente estator A : 0x4602
  - Torque electromecanico Nm : 0x4602
  - Velocidad RPM : 0x606C
4.   COB-ID: $0x00000300$
  - Iq objetivo A : $0x4600,2$
  - Iq A : $0x4600,8$
  - Demanda de Torque A : $0x4602,11$
  - Uq V : $0x4600,10$
5.   COB-ID: 0x00000400
  - Voltaje acelerador V : $0x2220,0$
  - Id A : $0x460,7$
  - Id objetivo A : $0x4600,5$
  - Demanda de Torque V : $0x4602,11$


# RPDO





Maestro
1.   COB-ID: $0x00000201$
  - Status word right motor: $0x2021,2$
  - Valor Actual Velocidad right motor RPM : $0x606C,0$
  - Torque Actual right motor Nm : $0x6077,0$

Esclavo

1.   COB-ID: $0x00000181$
  - Controlword : $0x6040,0$
  - Velocidad Máxima motor RPM : $0x6080,0$
  - Torque objetivo %: $0x6071,0$


# Función despliegue en línea

In [ ]:
"""
Librerias
"""
import can
import os
import time
from datetime import *
import numpy as np
# si el dato es positivo --> 00 00 00 00 + dato leido
# si el dato negativo --> FF FF FF FF - dato
"""
Diccionarios con parametros de TPDO para el Maestro y el esclavo
"""
sevcon1_Master = {
    "corrienteBat": 0,
    "VBateria": 0,
    "TemperaturaInv": 0,
    "Potencia_entrada": 0
}
sevcon2_Master = {
    "torque": 0,
    "velocidad": 0,
    "potencia_salida": 0,
    "corrienteInv": 0,
}
sevcon3_Master = {
    "targetIq": 0,
    "Iq": 0,
    "DemandTorque": 0,
    "Uq": 0,
}
sevcon4_Master = {
    "throttle_V": 0,
    "Id": 0,
    "targetId": 0,
    "Ud": 0,
}
sevcon1_Slave = {
    "corrienteBat": 0,
    "VBateria": 0,
    "TemperaturaInv": 0,
    "Potencia_entrada": 0
}
sevcon2_Slave = {
    "torque": 0,
    "velocidad": 0,
    "potencia_salida": 0,
    "corrienteInv": 0,
}
sevcon3_Slave = {
    "targetIq": 0,
    "Iq": 0,
    "DemandTorque": 0,
    "Uq": 0,
}
sevcon4_Slave = {
    "throttle_V": 0,
    "Id": 0,
    "targetId": 0,
    "Ud": 0,
}
"""
Factores de correcion
"""
sf = {
    "motor_RPM": 1,
    "motor_C": 1,
    "inverter_I": 1,
    "battery_V": 0.0625,
    "battery_C": 0.0625,
    "motor_torque": 0.1,
    "throttle_V": 0.00390625, 
    "IUqd V": 0.0625,
    "Current A(RMS)": 1
}
"""
COB-ID de TPDO para Maestro y Esclavo
"""
TPDOMaster = {
    "TPDO1_idM": 100,
    "TPDO2_idM": 200,
    "TPDO3_idM": 300,
    "TPDO4_idM": 400
}
TPDOSlave = {
    "TPDO1_idS": 102,
    "TPDO2_idS": 202,
    "TPDO3_idS": 302,
    "TPDO4_idS": 402
}

def twos_comp(val, bits):
    """
    Calculo del complemento de dos de un entero
    """
    if (val & (1 << (bits - 1))) != 0: # if sign bit is set e.g., 8bit: 128-255
        val = val - (1 << bits)        # compute negative value
    return val                         # return positive value as is

def m_hex(num):
    """
    Funcion covertir a hex
    """
    return hex(num)[2:]

def parse_msg(data, start, end):
    """
    Funcion segmentacion de a trama en red CANopen
    """
    msg = ""
    end += 1
    for i in range(start, end):
        msg = m_hex(data[i]) + msg
    # return int(msg, 16)
    return twos_comp(int(msg, 16), (end - start) * 8)

def read_messages(bus, TPDO, s):
    """
    Funcion segmentacion de a trama en red CANopen
    """
   global sevcon1_Master, sevcon2_Master, sevcon3_Master, sevcon4_Master, sevcon1_Slave, sevcon2_Slave, sevcon3_Slave, sevcon4_Slave 
   for msg in bus:
        data = msg.data
        msg = str(msg)
        cod_id = int(msg[39:45])
        
        # Master
        if cod_id == 200: 
            corriente = parse_msg(data, 0, 1) * sf["motor_C"] 
            torque_actual = parse_msg(data, 2, 3) * sf["motor_torque"]
            # if torque_actual >= 85:
            #     torque_actual = -(int("FFFF", 16) - int("FFFF", 16) +
            #                       4096 - torque_actual)
            RPM = parse_msg(data, 4, 7) * sf["motor_RPM"]
            # if RPM >= 900:
            #     RPM = -(int("FFFFFFFF",16) - RPM)
            w = (RPM * 2 * np.pi / 60) # velocidad angular
            Pout = torque_actual * w
            sevcon2_Master["torque"] = torque_actual
            sevcon2_Master["potencia_salida"] = Pout
            sevcon2_Master["velocidad"] = w
            sevcon2_Master["corriente"] = corriente
        if cod_id == 100:
            battery_V = parse_msg(data, 0, 1) * sf["battery_V"]
            battery_C = parse_msg(data, 2, 3) * sf["battery_C"]
            # if battery_C >= 4095:
            #     battery_C = 0
            inverter_temp = parse_msg(data, 4, 4) * sf["inverter_I"]
            sevcon1_Master["TemperaturaInv"] = inverter_temp
            sevcon1_Master["VBateria"] = battery_V
            sevcon1_Master["corrienteBat"] = battery_C
            sevcon1_Master["Potencia_entrada"] = battery_C * battery_V
        if cod_id == 300:
            Tiq = parse_msg(data, 0, 1) * sf["IUqd V"]
            iq = parse_msg(data, 2, 3) * sf["IUqd V"]
            torque_demand = parse_msg(data, 4, 5) * sf["motor_torque"]
            # if torque_demand >= 85:
            #     torque_demand = -(int("FFFF",16) - int("FFFF", 16) +
            #                       4096 - torque_demand)
            Uq = parse_msg(data, 6, 7) * sf["IUqd V"]
            sevcon3_Master["targetIq"] = Tiq
            sevcon3_Master["Iq"] = iq
            sevcon3_Master["DemandTorque"] = torque_demand
            sevcon3_Master["Uq"] = Uq
        if cod_id == 400:
            throttle_V = parse_msg(data, 0, 1) * sf["motor_C"] 
            id = parse_msg(data, 2, 3) * sf["IUqd V"]
            Tid = parse_msg(data, 4, 5) * sf["IUqd V"]
            Ud = parse_msg(data, 6, 7) * sf["IUqd V"]
            sevcon4_Master["throttle_V"] = throttle_V
            sevcon4_Master["Ud"] = Ud
            sevcon4_Master["targetId"] = Tid
            sevcon4_Master["Id"] = id

            # Slave
        if cod_id == 202: 
            corriente = parse_msg(data, 0, 1) * sf["motor_C"] 
            torque_actual = parse_msg(data, 2, 3) * sf["motor_torque"]
            # if torque_actual >= 85:
            #     torque_actual = -(int("FFFF", 16) - int("FFFF", 16) +
            #                       4096 - torque_actual)
            RPM = parse_msg(data, 4, 7) * sf["motor_RPM"]
            # if RPM >= 900:
            #     RPM = -(int("FFFFFFFF",16) - RPM)
            w = (RPM * 2 * np.pi / 60) # velocidad angular
            Pout = torque_actual * w
            sevcon2_Slave["torque"] = torque_actual
            sevcon2_Slave["potencia_salida"] = Pout
            sevcon2_Slave["velocidad"] = w
            sevcon2_Slave["corriente"] = corriente
        if cod_id == 102:
            battery_V = parse_msg(data, 0, 1) * sf["battery_V"]
            battery_C = parse_msg(data, 2, 3) * sf["battery_C"]
            # if battery_C >= 4095:
            #     battery_C = 0
            inverter_temp = parse_msg(data, 4, 4) * sf["inverter_I"]
            sevcon1_Slave["TemperaturaInv"] = inverter_temp
            sevcon1_Slave["VBateria"] = battery_V
            sevcon1_Slave["corrienteBat"] = battery_C
            sevcon1_Slave["Potencia_entrada"] = battery_C * battery_V
        if cod_id == 302:
            Tiq = parse_msg(data, 0, 1) * sf["IUqd V"]
            iq = parse_msg(data, 2, 3) * sf["IUqd V"]
            torque_demand = parse_msg(data, 4, 5) * sf["motor_torque"]
            # if torque_demand >= 85:
            #     torque_demand = -(int("FFFF",16) - int("FFFF", 16) +
            #                       4096 - torque_demand)
            Uq = parse_msg(data, 6, 7) * sf["IUqd V"]
            sevcon3_Slave["targetIq"] = Tiq
            sevcon3_Slave["Iq"] = iq
            sevcon3_Slave["DemandTorque"] = torque_demand
            sevcon3_Slave["Uq"] = Uq
        if cod_id == 402:
            throttle_V = parse_msg(data, 0, 1) * sf["motor_C"] 
            id = parse_msg(data, 2, 3) * sf["IUqd V"]
            Tid = parse_msg(data, 4, 5) * sf["IUqd V"]
            Ud = parse_msg(data, 6, 7) * sf["IUqd V"]
            sevcon4_Slave["throttle_V"] = throttle_V
            sevcon4_Slave["Ud"] = Ud
            sevcon4_Slave["targetId"] = Tid
            sevcon4_Slave["Id"] = id

def despliegueTPDO(ID):
    """
    Funcion segmentacion de a trama en red CANopen
    """
    
    """
    Master
    """
    if ID == 100:
        print('Maestro COB-ID: ' + str(ID))
        print("torque: {}, velocidad: {} , potencia: {}, corriente: {}".format(sevcon1_Master["torque"],
                                                                               sevcon1_Master["velocidad"], 
                                                                               sevcon1_Master["Potencia_salida"], 
                                                                               sevcon1_Master["corriente"]))
    elif ID == 200:
        print('Maestro COB-ID: ' + str(ID))
        print("TemperaturaInv: {} , VBateria: {} , IBateria: {}, Pin: {}".format(sevcon2_Master["TemperaturaInv"], 
                                                                                 sevcon2_Master["VBateria"], 
                                                                                 sevcon2_Master["IBateria"], 
                                                                                 sevcon2_Master["Potencia_entrada"]))
    elif ID == 300:
        print('Maestro COB-ID: ' + str(ID))
        print("Target Id: {} , Id: {} , DemandTorq: {} , Uq: {}".format(sevcon3_Master["targetIq"],
                                                                        sevcon3_Master["Iq"],
                                                                        sevcon3_Master["DemandTorque"],
                                                                        sevcon3_Master["Uq"]))
    elif ID == 400:
        print('Maestro COB-ID: ' + str(ID))
        print("Acelerador: {} , Id: {} , Ud: {} , targetId: {}".format(sevcon4_Master["throttle_V"],
                                                                       sevcon4_Master["Id"], 
                                                                       sevcon4_Master["Ud"], 
                                                                       sevcon4_Master["targetId"]))
    """
    Slave
    """
    elif ID == 102:
        print('Esclavo COB-ID: ' + str(ID))
        print("torque: {}, velocidad: {} , potencia: {}, corriente: {}".format(sevcon1_Slave["torque"],
                                                                               sevcon1_Slave["velocidad"], 
                                                                               sevcon1_Slave["potencia_salida"], 
                                                                               sevcon1_Slave["corriente"]))
    elif ID == 202:
        print('Esclavo COB-ID: ' + str(ID))
        print("TemperaturaInv: {} , VBateria: {} , IBateria: {}, Pin: {}".format(sevcon2_Slave["TemperaturaInv"], 
                                                                                 sevcon2_Slave["VBateria"], 
                                                                                 sevcon2_Slave["IBateria"], 
                                                                                 sevcon2_Slave["Potencia_entrada"]))
    elif ID == 302:
        print('Esclavo COB-ID: ' + str(ID))
        print("Target Id: {} , Id: {} , DemandTorq: {} , Uq: {}".format(sevcon3_Slave["targetIq"],
                                                                        sevcon3_Slave["Iq"],
                                                                        sevcon3_Slave["DemandTorque"],
                                                                        sevcon3_Slave["Uq"]))
    elif ID == 402:
        print('Esclavo COB-ID: ' + str(ID))
        print("Acelerador: {} , Id: {} , Ud: {} , targetId: {}".format(sevcon4_Slave["throttle_V"],
                                                                       sevcon4_Slave["Id"], 
                                                                       sevcon4_Slave["Ud"], 
                                                                       sevcon4_Slave["targetId"]))
    else:
        print('COB-ID no existente')
        break

# Main driver
if __name__ == '__main__':
    '''
    Set up
    '''
    # Request
    print('Indique COB-ID')
    id  = input()
    despliegueTPDO(id)
    # Despliegue
    print('\n\rCAN Rx test')
    print('Bring up CAN1....')
    os.system("sudo /sbin/ip link set can0 up type can bitrate 1000000")
    try:
      bus = can.interface.Bus(channel='can0', bustype='socketcan_native')
    except OSError:
      print('Cannot find PiCAN board.')
      exit()
    print('Ready')
    try:
        while True:
            read_messages(bus, TPDO, sf)
            printTPDO(id)
    except KeyboardInterrupt:
      #Catch keyboard interrupt
      os.system("sudo /sbin/ip link set can0 down")
      print('\n\rKeyboard interrtupt')

# Función Gráfica en línea

In [ ]:
"""
Librerias
"""
import can
import os
import time
from datetime import *
import numpy as np
# si el dato es positivo --> 00 00 00 00 + dato leido
# si el dato negativo --> FF FF FF FF - dato
"""
Diccionarios con parametros de TPDO para el Maestro y el esclavo
"""
sevcon1_Master = {
    "corrienteBat": 0,
    "VBateria": 0,
    "TemperaturaInv": 0,
    "Potencia_entrada": 0
}
sevcon2_Master = {
    "torque": 0,
    "velocidad": 0,
    "potencia_salida": 0,
    "corrienteInv": 0,
}
sevcon3_Master = {
    "targetIq": 0,
    "Iq": 0,
    "DemandTorque": 0,
    "Uq": 0,
}
sevcon4_Master = {
    "throttle_V": 0,
    "Id": 0,
    "targetId": 0,
    "Ud": 0,
}
sevcon1_Slave = {
    "corrienteBat": 0,
    "VBateria": 0,
    "TemperaturaInv": 0,
    "Potencia_entrada": 0
}
sevcon2_Slave = {
    "torque": 0,
    "velocidad": 0,
    "potencia_salida": 0,
    "corrienteInv": 0,
}
sevcon3_Slave = {
    "targetIq": 0,
    "Iq": 0,
    "DemandTorque": 0,
    "Uq": 0,
}
sevcon4_Slave = {
    "throttle_V": 0,
    "Id": 0,
    "targetId": 0,
    "Ud": 0,
}
"""
Factores de correcion
"""
sf = {
    "motor_RPM": 1,
    "motor_C": 1,
    "inverter_I": 1,
    "battery_V": 0.0625,
    "battery_C": 0.0625,
    "motor_torque": 0.1,
    "throttle_V": 0.00390625, 
    "IUqd V": 0.0625,
    "Current A(RMS)": 1
}
"""
COB-ID de TPDO para Maestro y Esclavo
"""
TPDOMaster = {
    "TPDO1_idM": 100,
    "TPDO2_idM": 200,
    "TPDO3_idM": 300,
    "TPDO4_idM": 400
}
TPDOSlave = {
    "TPDO1_idS": 102,
    "TPDO2_idS": 202,
    "TPDO3_idS": 302,
    "TPDO4_idS": 402
}

def twos_comp(val, bits):
    """
    Calculo del complemento de dos de un entero
    """
    if (val & (1 << (bits - 1))) != 0: # if sign bit is set e.g., 8bit: 128-255
        val = val - (1 << bits)        # compute negative value
    return val                         # return positive value as is

def m_hex(num):
    """
    Funcion covertir a hex
    """
    return hex(num)[2:]
def parse_msg(data, start, end):
    """
    Funcion segmentacion de a trama en red CANopen
    """
    msg = ""
    end += 1
    for i in range(start, end):
        msg = m_hex(data[i]) + msg
    # return int(msg, 16)
    return twos_comp(int(msg, 16), (end - start) * 8)
def read_messages(bus, TPDO, s):
    """
    Funcion segmentacion de a trama en red CANopen
    """
   global sevcon1_Master, sevcon2_Master, sevcon3_Master, sevcon4_Master, sevcon1_Slave, sevcon2_Slave, sevcon3_Slave, sevcon4_Slave 
   for msg in bus:
        data = msg.data
        msg = str(msg)
        cod_id = int(msg[39:45])
        
        # Master
        if cod_id == 200: 
            corriente = parse_msg(data, 0, 1) * sf["motor_C"] 
            torque_actual = parse_msg(data, 2, 3) * sf["motor_torque"]
            # if torque_actual >= 85:
            #     torque_actual = -(int("FFFF", 16) - int("FFFF", 16) +
            #                       4096 - torque_actual)
            RPM = parse_msg(data, 4, 7) * sf["motor_RPM"]
            # if RPM >= 900:
            #     RPM = -(int("FFFFFFFF",16) - RPM)
            w = (RPM * 2 * np.pi / 60) # velocidad angular
            Pout = torque_actual * w
            sevcon2_Master["torque"] = torque_actual
            sevcon2_Master["potencia_salida"] = Pout
            sevcon2_Master["velocidad"] = w
            sevcon2_Master["corriente"] = corriente
        if cod_id == 100:
            battery_V = parse_msg(data, 0, 1) * sf["battery_V"]
            battery_C = parse_msg(data, 2, 3) * sf["battery_C"]
            # if battery_C >= 4095:
            #     battery_C = 0
            inverter_temp = parse_msg(data, 4, 4) * sf["inverter_I"]
            sevcon1_Master["TemperaturaInv"] = inverter_temp
            sevcon1_Master["VBateria"] = battery_V
            sevcon1_Master["corrienteBat"] = battery_C
            sevcon1_Master["Potencia_entrada"] = battery_C * battery_V
        if cod_id == 300:
            Tiq = parse_msg(data, 0, 1) * sf["IUqd V"]
            iq = parse_msg(data, 2, 3) * sf["IUqd V"]
            torque_demand = parse_msg(data, 4, 5) * sf["motor_torque"]
            # if torque_demand >= 85:
            #     torque_demand = -(int("FFFF",16) - int("FFFF", 16) +
            #                       4096 - torque_demand)
            Uq = parse_msg(data, 6, 7) * sf["IUqd V"]
            sevcon3_Master["targetIq"] = Tiq
            sevcon3_Master["Iq"] = iq
            sevcon3_Master["DemandTorque"] = torque_demand
            sevcon3_Master["Uq"] = Uq
        if cod_id == 400:
            throttle_V = parse_msg(data, 0, 1) * sf["motor_C"] 
            id = parse_msg(data, 2, 3) * sf["IUqd V"]
            Tid = parse_msg(data, 4, 5) * sf["IUqd V"]
            Ud = parse_msg(data, 6, 7) * sf["IUqd V"]
            sevcon4_Master["throttle_V"] = throttle_V
            sevcon4_Master["Ud"] = Ud
            sevcon4_Master["targetId"] = Tid
            sevcon4_Master["Id"] = id

            # Slave
        if cod_id == 202: 
            corriente = parse_msg(data, 0, 1) * sf["motor_C"] 
            torque_actual = parse_msg(data, 2, 3) * sf["motor_torque"]
            # if torque_actual >= 85:
            #     torque_actual = -(int("FFFF", 16) - int("FFFF", 16) +
            #                       4096 - torque_actual)
            RPM = parse_msg(data, 4, 7) * sf["motor_RPM"]
            # if RPM >= 900:
            #     RPM = -(int("FFFFFFFF",16) - RPM)
            w = (RPM * 2 * np.pi / 60) # velocidad angular
            Pout = torque_actual * w
            sevcon2_Slave["torque"] = torque_actual
            sevcon2_Slave["potencia_salida"] = Pout
            sevcon2_Slave["velocidad"] = w
            sevcon2_Slave["corriente"] = corriente
        if cod_id == 102:
            battery_V = parse_msg(data, 0, 1) * sf["battery_V"]
            battery_C = parse_msg(data, 2, 3) * sf["battery_C"]
            # if battery_C >= 4095:
            #     battery_C = 0
            inverter_temp = parse_msg(data, 4, 4) * sf["inverter_I"]
            sevcon1_Slave["TemperaturaInv"] = inverter_temp
            sevcon1_Slave["VBateria"] = battery_V
            sevcon1_Slave["corrienteBat"] = battery_C
            sevcon1_Slave["Potencia_entrada"] = battery_C * battery_V
        if cod_id == 302:
            Tiq = parse_msg(data, 0, 1) * sf["IUqd V"]
            iq = parse_msg(data, 2, 3) * sf["IUqd V"]
            torque_demand = parse_msg(data, 4, 5) * sf["motor_torque"]
            # if torque_demand >= 85:
            #     torque_demand = -(int("FFFF",16) - int("FFFF", 16) +
            #                       4096 - torque_demand)
            Uq = parse_msg(data, 6, 7) * sf["IUqd V"]
            sevcon3_Slave["targetIq"] = Tiq
            sevcon3_Slave["Iq"] = iq
            sevcon3_Slave["DemandTorque"] = torque_demand
            sevcon3_Slave["Uq"] = Uq
        if cod_id == 402:
            throttle_V = parse_msg(data, 0, 1) * sf["motor_C"] 
            id = parse_msg(data, 2, 3) * sf["IUqd V"]
            Tid = parse_msg(data, 4, 5) * sf["IUqd V"]
            Ud = parse_msg(data, 6, 7) * sf["IUqd V"]
            sevcon4_Slave["throttle_V"] = throttle_V
            sevcon4_Slave["Ud"] = Ud
            sevcon4_Slave["targetId"] = Tid
            sevcon4_Slave["Id"] = id
        
fig1, axs = plt.subplots(1, 3)
y1, y2, y3, y4 = [],[],[],[]
y12, y22, y32, y42 = [],[],[],[] 
axs[0].set_title('RPM', fontsize=15)
axs[0].grid(True)
axs[1].set_title('bat_V', fontsize=15)
axs[1].grid(True)
axs[2].set_title('bat_I', fontsize=15)
axs[2].grid(True)

def draw_graph1(i):
    '''hex'''
    global count
    d0, d1, d2, d3, d5 = read_messages(bus, TPDO, sf)
    if d0 == 100:
        y1.append(d0)
        y2.append(d1)
        y3.append(d2)
        y4.append(d3)
#        plt.cla()
        #ax1.plot(y2, '*-')
        axs[0].plot(y2, '*-')
        #ax2.plot(y3, '*-')
        axs[1].plot(y3, '*-')
        #ax3.plot(y4, '*-')
        axs[2].plot(y4, '*-')

print('\n\rCAN Rx test')
print('Bring up CAN1....')
os.system("sudo /sbin/ip link set can0 up type can bitrate 1000000")
bus = can.interface.Bus(channel='can0', bustype='socketcan_native')
print('Ready')

ani = animation.FuncAnimation(plt.gcf(), draw_graph1)
plt.show()